In [195]:
import os
import json
import pandas as pd
import traceback

In [196]:
from langchain.chat_models import ChatOpenAI


In [197]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [198]:
KEY=os.getenv("OPENAI_API_KEY")

In [199]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [200]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [201]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [202]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [203]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [204]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [205]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [206]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [207]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [208]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [209]:
file_path=r"/Users/user/code/genai/GenAIMCQ/data.txt"

In [210]:
file_path

'/Users/user/code/genai/GenAIMCQ/data.txt'

In [211]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [212]:
print(TEXT)

Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large Language Models and its applications
Multi-modal Generative AI and its applications
Gen AI based Plugins and

In [213]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [214]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"


In [215]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large 

In [216]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1360
Prompt Tokens:937
Completion Tokens:423
Total Cost:0.0022515


In [217]:
response

{'text': 'Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:\n\nSystems, architecture and infrastructure for Generative AI\nMachine learning and Modeling using LLMs and Diffusion models\nLarge Language Models and its applications\nMulti-modal Generative AI and its applications\nGen AI b

In [218]:
quiz=response.get("quiz")

In [219]:
quiz=json.loads(quiz)

In [220]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [221]:
quiz_table_data

[{'MCQ': 'What is the focus of the workshop on Generative AI at AIMLSystems?',
  'Choices': 'a: Building robots | b: Creating realistic text, images, and video | c: Exploring outer space | d: Studying ancient civilizations',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a topic of interest for the workshop on Generative AI?',
  'Choices': 'a: Systems, architecture, and infrastructure for Generative AI | b: Machine learning and Modeling using LLMs and Diffusion models | c: Study of marine biology | d: Responsible use of Gen AI',
  'Correct': 'c'},
 {'MCQ': 'What is one of the applications enabled by Large Language Models according to the text?',
  'Choices': 'a: Creating music | b: Developing video games | c: Building robots | d: Generating realistic text',
  'Correct': 'd'},
 {'MCQ': 'What is one of the key concerns associated with the deployment of Generative AI solutions?',
  'Choices': 'a: Fairness, transparency, and accountability | b: Efficiency of solar panels | c: S

In [222]:
quiz=pd.DataFrame(quiz_table_data)

In [223]:
quiz.to_csv("machinelearning.csv",index=False)

In [224]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_27_2024_13_28_10'